In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score, mean_squared_log_error
import joblib
from osgeo import gdal, osr
import time
# import rasterio

In [ ]:
# Load TIFF images and extract data
# def load_tiff_data(file_path):
#     load_time=time.time()
#     src_ds = gdal.Open(file_path)
#     data = src_ds.ReadAsArray()
#     metadata = src_ds.GetGeoTransform(), src_ds.GetProjection()
#     src_ds = None  # Close dataset
#     end_load_time=time.time()
#     loading_time=end_load_time-load_time
#     print('loading_time:', loading_time)
#     print('the dimension of input:',data.shape)
#     return data, metadata



# Function to load data from .tif files
def load_tiff_data(file_path):
    with rasterio.open(file_path) as src:
        data = src.read()  # Read the first band
    return data
temperature = load_tiff_data(r"E:aligned_to_dmi_temp_2025_2050_250m.tif") # path1
rainfall = load_tiff_data(r"E:\aligned_to_dmi_rain_2025_2050_250m.tif") # path2
# pet = load_tiff_data('G:/Trend_Analysis_Data_Yearly/Barmer_Zone/AET_3_IA.tif')
# soil_moisture= load_tiff_data('G:/Trend_Analysis_Data_Yearly/Barmer_Zone/SoilMoisture_3_IA.tif')
# ndvi = load_tiff_data('G:/Trend_Analysis_Data_Yearly/Barmer_Zone/NDVI_3_IA.tif')

In [ ]:
print('dimension of temperature',temperature.shape)

In [ ]:
loaded_model = joblib.load('random_forest_model_agr_ddi_100trees_5depth.pkl')

In [ ]:
# temperature = load_tiff_data('G:/Trend_Analysis_Data_Yearly/Barmer_Zone/LST_3_IA.tif')
# rainfall = load_tiff_data('G:/Trend_Analysis_Data_Yearly/Barmer_Zone/Rainfall_3_IA.tif')
# pet = load_tiff_data('G:/Trend_Analysis_Data_Yearly/Barmer_Zone/AET_3_IA.tif')
# soil_moisture= load_tiff_data('G:/Trend_Analysis_Data_Yearly/Barmer_Zone/SoilMoisture_3_IA.tif')
# ndvi = load_tiff_data('G:/Trend_Analysis_Data_Yearly/Barmer_Zone/NDVI_3_IA.tif')

# Prepare the data
temperature_flat = temperature.flatten()
rainfall_flat = rainfall.flatten()
# pet_flat = pet.flatten()
# soil_moisture_flat = soil_moisture.flatten()
# ndvi_flat = ndvi.flatten()

data = {
    'temperature': temperature_flat,
    'rainfall': rainfall_flat,
    # 'pet': pet_flat,
    # 'soil_moisture': soil_moisture_flat,
    # 'ndvi': ndvi_flat
}

df = pd.DataFrame(data)
# Convert reshaped image to DataFrame

# Check for NaN values
nan_columns = df.columns[df.isna().any()].tolist()
print("Columns with NaN values:", nan_columns)

# Mask NaN values with mean
df.fillna(df.mean(), inplace=True)

# Convert DataFrame back to numpy array
reshaped_image_handled = df.to_numpy()
print(reshaped_image_handled.shape)
X=reshaped_image_handled

# Drop NaN values if any
# df.dropna(inplace=True)

# Features and target variable
# X = df[['temperature', 'rainfall', 'pet', 'soil_moisture']]
X = df[['temperature', 'rainfall']]
X1=X.shape
# y_test = df['ndvi']
# print(data)

start_of_prediction=time.time()
y_pred = loaded_model.predict(X)
end_of_prediction=time.time()
prediction_diff=start_of_prediction-end_of_prediction
print(prediction_diff/3600)
# mse = mean_squared_error(y_test, y_pred)
# rmse = np.sqrt(mse)
# mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)
# explained_variance = explained_variance_score(y_test, y_pred)
# Print the evaluation metrics for random forest on the test set
# print("\nEvaluation metrics for rf on the test set:")
# print("Mean squared error (MSE):", mse)
# print("Root mean squared error (RMSE):", rmse)
# print("Mean absolute error (MAE):", mae)
# print("R-squared (R2):", r2)
# print("Explained variance score:", explained_variance)


## Prediction for 26 years (Debugged)

In [ ]:
from osgeo import gdal, osr
# Open the input image using GDAL to get its metadata
input_image_path = (r'E:\aligned_to_dmi_temp_2025_2050_250m.tif')
input_dataset = gdal.Open(input_image_path)
metadata = input_dataset.GetMetadata()
print(metadata)
# Get the metadata from the input image
geotransform = input_dataset.GetGeoTransform()
print(geotransform)
srs = osr.SpatialReference()
srs.ImportFromEPSG(4326)
projection= srs.ExportToWkt()
print(projection)
num_rows = input_dataset.RasterYSize
num_cols = input_dataset.RasterXSize
num_years=26
# Close the input dataset
input_dataset = None

# Reshape predicted NDVI values to match dimensions of input image
predicted_ndvi_reshaped = y_pred.reshape(num_years, num_rows, num_cols)

# Create new .tiff files for each year
for i in range(num_years):
    # Create new .tiff file
    output_path = f"ndvi_predicted_year_{i+1}.tif"
    driver = gdal.GetDriverByName("GTiff")
    dataset = driver.Create(output_path, num_cols, num_rows, 1, gdal.GDT_Float32)

    # Write predicted NDVI values to .tiff file
    dataset.GetRasterBand(1).WriteArray(predicted_ndvi_reshaped[i])

    # Set metadata (geotransform and projection) to match input image
    dataset.SetGeoTransform(geotransform)
    
    dataset.SetProjection(projection)

    # Close dataset
    dataset = None

    print(f"Predicted NDVI for year {i+1} saved as {output_path}")

In [ ]:
print(predicted_ndvi_reshaped.shape)